# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
# portfolio data example

portfolio.head()por

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [3]:
# profile data example

profile.head(10)

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN
5,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0
6,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN
7,118,20171002,None,68617ca6246f4fbc85e91a2a49552598,NaN
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0
9,118,20161122,None,8974fc5686fe429db53ddde067b88302,NaN


In [4]:
# strange looking profile data, further details
# what's age 118?  len(profile[(profile['age']==118)]) = 2175, probably null 
# profile118 = profile[(profile['age']==118)] (gender and income nulls too)
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [5]:
# transcript data example

transcript.tail()

,event,person,time,value
306529,transaction,b3a1272bc9904337b331bf348c3e8c17,714,{'amount': 1.5899999999999999}
306530,transaction,68213b08d99a4ae1b0dcb72aebd9aa35,714,{'amount': 9.53}
306531,transaction,a00058cf10334a308c68e7631c529907,714,{'amount': 3.61}
306532,transaction,76ddbd6576844afe811f1a3c0fbb5bec,714,{'amount': 3.5300000000000002}
306533,transaction,c02b10e8752c4d8e9b73f918558531f7,714,{'amount': 4.05}


TESTING WITH ONE PERSON

In [6]:
person = profile[profile['id']=='389bc3fa690240e798340f5a15918d5c']
person

,age,became_member_on,gender,id,income
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0


In [7]:
person_actions = transcript[transcript['person']=='389bc3fa690240e798340f5a15918d5c']
person_actions

,event,person,time,value
5,offer received,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
12650,offer viewed,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
32625,transaction,389bc3fa690240e798340f5a15918d5c,60,{'amount': 9.54}
32626,offer completed,389bc3fa690240e798340f5a15918d5c,60,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...
53179,offer received,389bc3fa690240e798340f5a15918d5c,168,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
77212,offer viewed,389bc3fa690240e798340f5a15918d5c,192,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
110831,offer received,389bc3fa690240e798340f5a15918d5c,336,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
132803,offer viewed,389bc3fa690240e798340f5a15918d5c,354,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
150601,offer received,389bc3fa690240e798340f5a15918d5c,408,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
179953,offer viewed,389bc3fa690240e798340f5a15918d5c,438,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}


In [8]:
person_offer = portfolio[(portfolio["id"].values =='2906b810c7d4411798c6938adc9daaa5') | (portfolio["id"].values == "f19421c1d4aa40978ebb69ca19b0e20d")]
person_offer

,channels,difficulty,duration,id,offer_type,reward
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [9]:
# create a new df with 'value' column dictionary opened to columns

transcript_new = pd.concat([transcript.drop(['value'], axis=1), transcript['value'].apply(pd.Series)], axis=1)

In [10]:
transcript_new.tail()

,event,person,time,offer id,amount,offer_id,reward
306529,transaction,b3a1272bc9904337b331bf348c3e8c17,714,NaN,1.59,NaN,NaN
306530,transaction,68213b08d99a4ae1b0dcb72aebd9aa35,714,NaN,9.53,NaN,NaN
306531,transaction,a00058cf10334a308c68e7631c529907,714,NaN,3.61,NaN,NaN
306532,transaction,76ddbd6576844afe811f1a3c0fbb5bec,714,NaN,3.53,NaN,NaN
306533,transaction,c02b10e8752c4d8e9b73f918558531f7,714,NaN,4.05,NaN,NaN


In [11]:
person_actions_new = transcript_new[transcript_new['person']=='389bc3fa690240e798340f5a15918d5c']
person_actions_new

,event,person,time,offer id,amount,offer_id,reward
5,offer received,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,NaN
12650,offer viewed,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,NaN
32625,transaction,389bc3fa690240e798340f5a15918d5c,60,NaN,9.54,NaN,NaN
32626,offer completed,389bc3fa690240e798340f5a15918d5c,60,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
53179,offer received,389bc3fa690240e798340f5a15918d5c,168,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
77212,offer viewed,389bc3fa690240e798340f5a15918d5c,192,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
110831,offer received,389bc3fa690240e798340f5a15918d5c,336,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
132803,offer viewed,389bc3fa690240e798340f5a15918d5c,354,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
150601,offer received,389bc3fa690240e798340f5a15918d5c,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
179953,offer viewed,389bc3fa690240e798340f5a15918d5c,438,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN


In [12]:
def replace_null_offer_id(data):
    data['offer id'] = data['offer id'].fillna(data['offer_id'])
    return data

replace_null_offer_id(transcript_new)
transcript_new.tail()

,event,person,time,offer id,amount,offer_id,reward
306529,transaction,b3a1272bc9904337b331bf348c3e8c17,714,NaN,1.59,NaN,NaN
306530,transaction,68213b08d99a4ae1b0dcb72aebd9aa35,714,NaN,9.53,NaN,NaN
306531,transaction,a00058cf10334a308c68e7631c529907,714,NaN,3.61,NaN,NaN
306532,transaction,76ddbd6576844afe811f1a3c0fbb5bec,714,NaN,3.53,NaN,NaN
306533,transaction,c02b10e8752c4d8e9b73f918558531f7,714,NaN,4.05,NaN,NaN


In [13]:
# drop offer_id
transcript_new = transcript_new.drop(columns=['offer_id'])

In [14]:
person_actions_new = transcript_new[transcript_new['person']=='389bc3fa690240e798340f5a15918d5c']
person_actions_new.sort_values(by=['time'])

,event,person,time,offer id,amount,reward
5,offer received,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
12650,offer viewed,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
32625,transaction,389bc3fa690240e798340f5a15918d5c,60,NaN,9.54,NaN
32626,offer completed,389bc3fa690240e798340f5a15918d5c,60,f19421c1d4aa40978ebb69ca19b0e20d,NaN,5.0
53179,offer received,389bc3fa690240e798340f5a15918d5c,168,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
77212,offer viewed,389bc3fa690240e798340f5a15918d5c,192,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
110831,offer received,389bc3fa690240e798340f5a15918d5c,336,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
132803,offer viewed,389bc3fa690240e798340f5a15918d5c,354,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
150601,offer received,389bc3fa690240e798340f5a15918d5c,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
179953,offer viewed,389bc3fa690240e798340f5a15918d5c,438,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN


In [15]:
events = pd.DataFrame(transcript_new['event'])
events_hot = pd.get_dummies(events)

In [16]:
events_hot.tail()

,event_offer completed,event_offer received,event_offer viewed,event_transaction
306529,0,0,0,1
306530,0,0,0,1
306531,0,0,0,1
306532,0,0,0,1
306533,0,0,0,1


In [17]:
# TEST
# one-hot-encode event column

test_df = pd.concat([transcript_new, pd.get_dummies(transcript_new['event'], prefix='event')], axis=1)
test_df.drop(['event'], axis=1, inplace=True)
test_df.tail()

,person,time,offer id,amount,reward,event_offer completed,event_offer received,event_offer viewed,event_transaction
306529,b3a1272bc9904337b331bf348c3e8c17,714,NaN,1.59,NaN,0,0,0,1
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,NaN,9.53,NaN,0,0,0,1
306531,a00058cf10334a308c68e7631c529907,714,NaN,3.61,NaN,0,0,0,1
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,NaN,3.53,NaN,0,0,0,1
306533,c02b10e8752c4d8e9b73f918558531f7,714,NaN,4.05,NaN,0,0,0,1


In [18]:
# person hot encoded

person = test_df[test_df['person']=='389bc3fa690240e798340f5a15918d5c']
person_hot = person[['person', 'offer id', 'event_offer received', 'event_offer viewed', 'event_offer completed', 'reward', 'amount', 'time']]
person_hot.sort_values(by=['offer id', 'event_offer received', 'event_offer viewed', 'event_offer completed'], ascending=False)

,person,offer id,event_offer received,event_offer viewed,event_offer completed,reward,amount,time
245126,389bc3fa690240e798340f5a15918d5c,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,576
265621,389bc3fa690240e798340f5a15918d5c,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,NaN,NaN,588
271782,389bc3fa690240e798340f5a15918d5c,fafdcd668e3743c1bb461111dcafc2a4,0,0,1,2.0,NaN,600
5,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,NaN,NaN,0
201577,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,NaN,NaN,504
12650,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,1,0,NaN,NaN,0
214275,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,1,0,NaN,NaN,504
32626,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,0,1,5.0,NaN,60
271781,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,0,1,5.0,NaN,600
53179,389bc3fa690240e798340f5a15918d5c,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,NaN,NaN,168


In [20]:
# person_hot grouped and sorted

aggregation_functions = {
    'person': 'first',
    'event_offer completed': 'sum', 
    'event_offer received': 'sum', 
    'event_offer viewed': 'sum',
    'time': 'first',
    'amount': 'sum',
    'reward': 'sum'}
person_grouped = person_hot.groupby(person_hot['offer id']).aggregate(aggregation_functions)

In [21]:
person_grouped_sorted = person_grouped[['person', 'event_offer received', 'event_offer viewed', 'event_offer completed', 'reward', 'amount', 'time']]
person_grouped_sorted.sort_values(by=['event_offer received', 'event_offer viewed', 'event_offer completed'], ascending=False)

,person,event_offer received,event_offer viewed,event_offer completed,reward,amount,time
offer id,,,,,,,
f19421c1d4aa40978ebb69ca19b0e20d,389bc3fa690240e798340f5a15918d5c,2,2,2,10.0,0.0,0
9b98b8c7a33c4b65b9aebfe6a799e6d9,389bc3fa690240e798340f5a15918d5c,2,2,1,5.0,0.0,168
2906b810c7d4411798c6938adc9daaa5,389bc3fa690240e798340f5a15918d5c,1,1,1,2.0,0.0,336
fafdcd668e3743c1bb461111dcafc2a4,389bc3fa690240e798340f5a15918d5c,1,1,1,2.0,0.0,576


In [22]:
offer_details = portfolio[(portfolio["id"].values =='f19421c1d4aa40978ebb69ca19b0e20d') 
                 | (portfolio["id"].values == '9b98b8c7a33c4b65b9aebfe6a799e6d9')
                 | (portfolio["id"].values == '2906b810c7d4411798c6938adc9daaa5')
                 | (portfolio["id"].values == 'fafdcd668e3743c1bb461111dcafc2a4')]
offer_details

,channels,difficulty,duration,id,offer_type,reward
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [27]:
offer_details.set_index('id', inplace=True)
offer_details

,channels,difficulty,duration,offer_type,reward
id,,,,,
9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5
fafdcd668e3743c1bb461111dcafc2a4,"[web, email, mobile, social]",10,10,discount,2
f19421c1d4aa40978ebb69ca19b0e20d,"[web, email, mobile, social]",5,5,bogo,5
2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]",10,7,discount,2


In [38]:
#offer_details_hot = pd.DataFrame(offer_details['channels'].values.tolist(), columns=['web','email', 'mobile', 'social'])

#one hot encoding
offer_cols = pd.get_dummies(offer_details[['offer_type']]) 
offer_details_hot = pd.concat([offer_details, offer_cols], axis=1)

#parse channels list and create individual binary column for each channel
offer_details_hot['web'] = offer_details_hot['channels'].apply(lambda x: 1 if 'web' in x else 0)
offer_details_hot['email'] = offer_details_hot['channels'].apply(lambda x: 1 if 'email' in x else 0)
offer_details_hot['mobile'] = offer_details_hot['channels'].apply(lambda x: 1 if 'mobile' in x else 0)
offer_details_hot['social'] = offer_details_hot['channels'].apply(lambda x: 1 if 'social' in x else 0)

offer_details_hot.drop(['channels', 'offer_type'], inplace=True, axis=1)

offer_details_hot

,difficulty,duration,reward,offer_type_bogo,offer_type_discount,web,email,mobile,social
id,,,,,,,,,
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,7,5,1,0,1,1,1,0
fafdcd668e3743c1bb461111dcafc2a4,10,10,2,0,1,1,1,1,1
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,1,0,1,1,1,1
2906b810c7d4411798c6938adc9daaa5,10,7,2,0,1,1,1,1,0


### Transcript hot encode

In [23]:
transcript_hot = test_df[['person', 'offer id', 'event_offer received', 'event_offer viewed', 'event_offer completed', 'reward', 'amount', 'time']]
transcript_hot.sort_values(by=['offer id', 'event_offer received', 'event_offer viewed', 'event_offer completed'], ascending=False)

,person,offer id,event_offer received,event_offer viewed,event_offer completed,reward,amount,time
3,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
15,f806632c011441378d4646567f357a21,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
29,8be4463721e14d7fa600686bf8c8b2ed,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
34,9c64321646bf40f6ad1bc98864eaa6b8,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
59,816afd4505d047cf893275fbfe825590,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
63,9aba5058cb4f483da896335a1479a944,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
64,0ba6e2ad781c4429afd26cf0b35f2d11,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
65,53bac762af27471eb294ff89ca1bae2c,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
73,1abeb816d9fa49b4b5b5ae51bc3d24c3,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0
77,a9780228a1d04539afdee8b745e4be10,fafdcd668e3743c1bb461111dcafc2a4,1,0,0,NaN,NaN,0


In [24]:
aggregation_functions = {
    'event_offer completed': 'sum', 
    'event_offer received': 'sum', 
    'event_offer viewed': 'sum',
    'amount': 'sum',
    'reward': 'sum'}
transcript_grouped = transcript_hot.groupby(transcript_hot['offer id']).aggregate(aggregation_functions)
transcript_grouped.head(20)

,event_offer completed,event_offer received,event_offer viewed,amount,reward
offer id,,,,,
0b1e1539f2cc45b7b9fa7c272da2e1d7,3420.0,7668.0,2663.0,0.0,17100.0
2298d6c36e964ae4a3e7e9706d1fb8c2,5156.0,7646.0,7337.0,0.0,15468.0
2906b810c7d4411798c6938adc9daaa5,4017.0,7632.0,4118.0,0.0,8034.0
3f207df678b143eea3cee63160fa8bed,0.0,7617.0,4144.0,0.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,3331.0,7593.0,7298.0,0.0,33310.0
5a8bc65990b245e5a138643cd4eb9837,0.0,7618.0,6687.0,0.0,0.0
9b98b8c7a33c4b65b9aebfe6a799e6d9,4354.0,7677.0,4171.0,0.0,21770.0
ae264e3637204a6fb9bb56bc8210ddfd,3688.0,7658.0,6716.0,0.0,36880.0
f19421c1d4aa40978ebb69ca19b0e20d,4296.0,7571.0,7264.0,0.0,21480.0


In [25]:
transcript_hot_grouped = transcript_grouped[['event_offer received', 'event_offer viewed', 'event_offer completed', 'reward', 'amount']]
transcript_hot_grouped.sort_values(by=['event_offer received', 'event_offer viewed', 'event_offer completed'], ascending=False)

,event_offer received,event_offer viewed,event_offer completed,reward,amount
offer id,,,,,
9b98b8c7a33c4b65b9aebfe6a799e6d9,7677.0,4171.0,4354.0,21770.0,0.0
0b1e1539f2cc45b7b9fa7c272da2e1d7,7668.0,2663.0,3420.0,17100.0,0.0
ae264e3637204a6fb9bb56bc8210ddfd,7658.0,6716.0,3688.0,36880.0,0.0
2298d6c36e964ae4a3e7e9706d1fb8c2,7646.0,7337.0,5156.0,15468.0,0.0
2906b810c7d4411798c6938adc9daaa5,7632.0,4118.0,4017.0,8034.0,0.0
5a8bc65990b245e5a138643cd4eb9837,7618.0,6687.0,0.0,0.0,0.0
3f207df678b143eea3cee63160fa8bed,7617.0,4144.0,0.0,0.0,0.0
fafdcd668e3743c1bb461111dcafc2a4,7597.0,7327.0,5317.0,10634.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,7593.0,7298.0,3331.0,33310.0,0.0


In [26]:
# count of unique offer id's

print(transcript_new['offer id'].nunique())
transcript_new['offer id'].unique()

10


array(['9b98b8c7a33c4b65b9aebfe6a799e6d9',
       '0b1e1539f2cc45b7b9fa7c272da2e1d7',
       '2906b810c7d4411798c6938adc9daaa5',
       'fafdcd668e3743c1bb461111dcafc2a4',
       '4d5c57ea9a6940dd891ad53e9dbe8da0',
       'f19421c1d4aa40978ebb69ca19b0e20d',
       '2298d6c36e964ae4a3e7e9706d1fb8c2',
       '3f207df678b143eea3cee63160fa8bed',
       'ae264e3637204a6fb9bb56bc8210ddfd',
       '5a8bc65990b245e5a138643cd4eb9837', nan], dtype=object)